Part 1 question 2 
Group Id : ViDiSh 
Group Members : Divya Mishra, Shreyansh Aswal, Vipin Gautam

In [8]:
import sys
sys.path.append("C:\\Python39\\Lib\\site-packages")
import cv2 as cv
import numpy as np

image = cv.imread("C:\\Users\\user\\Downloads\\colors.jpg")
k=32
centroids=np.zeros(k*3).reshape(k,3)

#shape of the image is in the form (height, width, number_of_channels)
height = image.shape[0]
width = image.shape[1]
num_pixels=height*width


Assigning initial values to the 32 centroids

In [9]:
def initialize_centroids():
    j=1
    for i in range(k):
        centroids[i][0]=j
        centroids[i][1]=j+3
        centroids[i][2]=j+7
        j+=5


To Calculate the Cluster of a given pixel

In [10]:
def return_cluster(pixel):
    min=np.linalg.norm(centroids[0] - pixel)
    index=0
    for i in range(k):
        dist=np.linalg.norm(centroids[i] - pixel)
        #We calculate distance of each cluster's centroid from a given pixel, and assign it to the cluster having minimum distance from the pixel
        if dist < min:
            index=i
            min=dist
    return index

Applying K means Clustering to determine k = 32 centroids for a given image

In [11]:
def calc_cluster_each_pixel():
    i=0
    #Here, cluster array stores what cluster each pixel is a part of, so its size is equal to the total number of pixels in the image
    cluster=np.zeros(num_pixels)
    for row in image:
        for pixel in row:
            #Here k is the index of the cluster the current pixel is a part of
            k=return_cluster(pixel)
            cluster[i]=k
            i+=1
    return cluster

Re-computing the new Centroids based on the points in each cluster

In [12]:
def re_compute_centroids(cluster):
    centroids_old=centroids
    sum_r=np.zeros(32)
    sum_y=np.zeros(32)
    sum_b=np.zeros(32)
    count=np.zeros(32)
    i=0
    for row in image:
        for pixel in row:
            idx=int(cluster[i])
            #Since image is 3 channel, centroid would be a 3 feature vector
            sum_r[idx]+=pixel[0]
            sum_y[idx]+=pixel[1]
            sum_b[idx]+=pixel[2]
            count[idx]+=1
            i+=1

    for j in range(k):
        #computing centroid for each channel for a given cluster
        centroids[j][0]=sum_r[j]/count[j]
        centroids[j][1]=sum_y[j]/count[j]
        centroids[j][2]=sum_b[j]/count[j]
    
    #Here we return the old value of the centroids so that it can be compared with the current centroid, which is a global variable
    return centroids_old

Main Calling thread, here the centroids would be computed iteratively until the convergence condition is met

In [13]:
initialize_centroids()
while True:
    cluster=calc_cluster_each_pixel()
    centroids_old=re_compute_centroids(cluster)
    for j in range(k):
        diff=np.linalg.norm(centroids[j] - centroids_old[j])
    
    #convergence condition, when the distance between the old and new centroids is less than 0.1 units
    if diff <= 0.1:
        break

Compressing Image by replacing each pixel value with the corresponding centroid of the cluster it belongs to

In [14]:
i=0; 
for row in range(height):
    for pixel in range(width):
        c=int(cluster[i])
        image[row][pixel]=centroids[c]
        i+=1

cv.imshow('image', image)
cv.waitKey(0) 
  
cv.destroyAllWindows()         
